# Generate Customer Insight Report

In [1]:
import mlrun
import pandas as pd
from dotenv import load_dotenv
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import PromptTemplate
from langchain_openai.chat_models import ChatOpenAI

load_dotenv()

CHURN_FEATURES = [
    'international_plan',
    'voice_mail_plan',
    'sentiment_label',
    'account_length',
    'number_vmail_messages',
    'total_day_minutes',
    'total_day_calls',
    'total_day_charge',
    'total_eve_minutes',
    'total_eve_calls',
    'total_eve_charge',
    'total_night_minutes',
    'total_night_calls',
    'total_night_charge',
    'total_intl_minutes',
    'total_intl_calls',
    'total_intl_charge',
    'number_customer_service_calls',
    'sentiment_score'
]

USER_INFO_FEATURES = [
    "account_length",
    "total_charge_usage",
    "total_minutes_usage",
    "number_customer_service_calls",
    "sentiment_label"
]

SENTIMENT_MAPPING = {0 : "negative", 1: "neutral", 2: "positive"}

In [2]:
project = mlrun.get_or_create_project("churn-agents-webinar")

Project Source: v3io:///bigdata/churn.zip
Exporting project as zip archive to v3io:///bigdata/churn.zip...
> 2025-01-27 23:01:02,138 [info] Project loaded successfully: {"project_name":"churn-agents-webinar"}


### Load data

In [3]:
all_data = mlrun.get_dataitem("store://datasets/churn-agents-webinar/churn#0:latest").as_df()
test_data = mlrun.get_dataitem("store://datasets/churn-agents-webinar/data-process-data_test#0:latest").as_df()

In [4]:
all_data["total_charge"] = all_data["total_day_charge"] + all_data["total_eve_charge"] + all_data["total_night_charge"] + all_data["total_intl_charge"]
all_data["total_charge_usage"] = pd.qcut(all_data['total_charge'], q=5, labels=['low', 'medium-low', 'medium', 'medium-high', 'high'])

all_data["total_minutes"] = all_data["total_day_minutes"] + all_data["total_eve_minutes"] + all_data["total_night_minutes"] + all_data["total_intl_minutes"]
all_data["total_minutes_usage"] = pd.qcut(all_data['total_minutes'], q=5, labels=['low', 'medium-low', 'medium', 'medium-high', 'high'])

In [5]:
test_data = test_data.join(all_data[["total_charge_usage", "total_minutes_usage", "chat_log"]])

In [6]:
test_data.iloc[0]

international_plan                                                             0.0
voice_mail_plan                                                                0.0
sentiment_label                                                                1.0
account_length                                                                 117
number_vmail_messages                                                            0
total_day_minutes                                                            181.5
total_day_calls                                                                 95
total_day_charge                                                             30.86
total_eve_minutes                                                            205.1
total_eve_calls                                                                 88
total_eve_charge                                                             17.43
total_night_minutes                                                          204.0
tota

### Initialize ML and LLM models

In [7]:
serving_fn = project.get_function("serving")

llm = ChatOpenAI(name="gpt-4o-mini", temperature=0)
prompt = PromptTemplate(
    input_variables=["text"],
    template="Summarize the following support ticket into 1 or 2 sentences:\n\n{text}\n"
)
chain = prompt | llm | StrOutputParser()

### Generate report

In [8]:
def get_customer_insight_report(user_id: int) -> dict:
    
    def format_months(months):
        years = months // 12
        remaining_months = months % 12
        return f"{years} years, {remaining_months} months"
    
    def churn_inference(user_id):
        churn_features = test_data[CHURN_FEATURES].loc[user_id].tolist()
        resp = serving_fn.invoke(
            "/v2/models/churn_model/predict",
            body={
                "inputs": [churn_features]
            }
        )
        churn_pct = round(resp["outputs"][0], 3)
        return churn_pct
    
    def summarize_support_ticket(user_id):
        return chain.invoke({"text" : test_data.loc[user_id, "chat_log"]})
    
    def get_available_promotions(
        total_charge_usage: int,
        total_minutes_usage: int,
        number_customer_service_calls: int,
        sentiment_label: str,
        account_length_months: int,
        churn_likelihood_percentage: float,
        **kwargs
    ) -> list[str]:

        promotions = []

        if churn_likelihood_percentage > 0.6 or sentiment_label == "negative":
            promotions.append("Retention Offer: 20% discount for 6 months")
        if total_charge_usage in ["medium-high", "high"] or total_minutes_usage in ["medium-high", "high"]:
            promotions.append("Loyalty Offer: Free international minutes")
        if number_customer_service_calls >= 5:
            promotions.append("Service Credit: $10 off next bill")
        if sentiment_label == "positive" and churn_likelihood_percentage < 0.4:
            promotions.append("Upgrade Offer: Free device upgrade")
        if churn_likelihood_percentage <= 0.5 and total_charge_usage in ["medium", "medium-low"]:
            promotions.append("Bundle Offer: Add premium channels for $5/month")
        if account_length_months > 24:
            promotions.append("Anniversary Offer: 1 month free service")

        return promotions if promotions else ["General Offer: 10% discount for 3 months"]
    
    user_info = test_data.loc[user_id, USER_INFO_FEATURES]
    user_info["account_length_months"] = user_info["account_length"]
    user_info["account_length"] = format_months(user_info["account_length"])
    user_info["churn_likelihood_percentage"] = churn_inference(user_id)
    user_info["sentiment_label"] = SENTIMENT_MAPPING[user_info["sentiment_label"]]
    user_info["support_ticket_summary"] = summarize_support_ticket(user_id)
    user_info["available_promotions"] = get_available_promotions(**user_info.to_dict())
    user_info["user_id"] = user_id
    
    return user_info.to_dict()

### Test report generation

In [9]:
get_customer_insight_report(user_id=2296)

> 2025-01-27 23:01:02,817 [info] Invoking function: {"method":"POST","path":"http://nuclio-churn-agents-webinar-serving.default-tenant.svc.cluster.local:8080/v2/models/churn_model/predict"}


{'account_length': '9 years, 9 months',
 'total_charge_usage': 'medium',
 'total_minutes_usage': 'medium',
 'number_customer_service_calls': 2,
 'sentiment_label': 'neutral',
 'account_length_months': 117,
 'churn_likelihood_percentage': 0.005,
 'support_ticket_summary': 'The customer has voice, text, and data services and is looking to upgrade their phone due to it being slow for a while.',
 'available_promotions': ['Bundle Offer: Add premium channels for $5/month',
  'Anniversary Offer: 1 month free service'],
 'user_id': 2296}

In [10]:
get_customer_insight_report(user_id=460)

> 2025-01-27 23:01:03,427 [info] Invoking function: {"method":"POST","path":"http://nuclio-churn-agents-webinar-serving.default-tenant.svc.cluster.local:8080/v2/models/churn_model/predict"}


{'account_length': '6 years, 7 months',
 'total_charge_usage': 'medium-low',
 'total_minutes_usage': 'medium',
 'number_customer_service_calls': 5,
 'sentiment_label': 'neutral',
 'account_length_months': 79,
 'churn_likelihood_percentage': 0.483,
 'support_ticket_summary': 'The customer is concerned about the escalating cost of voice services impacting turnover and is looking for alternative options, but the TelCom agent offers free features instead, leading to the customer hanging up.',
 'available_promotions': ['Service Credit: $10 off next bill',
  'Bundle Offer: Add premium channels for $5/month',
  'Anniversary Offer: 1 month free service'],
 'user_id': 460}

In [12]:
get_customer_insight_report(user_id=1902)

> 2025-01-27 23:01:07,399 [info] Invoking function: {"method":"POST","path":"http://nuclio-churn-agents-webinar-serving.default-tenant.svc.cluster.local:8080/v2/models/churn_model/predict"}


{'account_length': '10 years, 5 months',
 'total_charge_usage': 'high',
 'total_minutes_usage': 'high',
 'number_customer_service_calls': 1,
 'sentiment_label': 'negative',
 'account_length_months': 125,
 'churn_likelihood_percentage': 0.848,
 'support_ticket_summary': 'Customer is frustrated with the poor customer service they have been receiving for their phone, despite not having any issues with the phone itself.',
 'available_promotions': ['Retention Offer: 20% discount for 6 months',
  'Loyalty Offer: Free international minutes',
  'Anniversary Offer: 1 month free service'],
 'user_id': 1902}